# 第6回課題

## 課題1

inorderでは左、自分、右の順番で木を探索する。

つまり、inorder(left(tree)) ,  value(tree) ,  inorder(right(tree))の順に再帰的に辿っていけばよい。


In [9]:
def make_node(num, left_node, right_node):
    return [num, left_node, right_node]

def value(tree):
    return tree[0]

def left(tree):
    return tree[1]

def right(tree):
    return tree[2]

EmptyTree = "None"

def is_empty(tree):
    return tree == EmptyTree

def make_leaf(num):
    return make_node(num, EmptyTree, EmptyTree)

def max_value(tree):
    if is_empty(tree):
        return float('-inf') 
    return max(value(tree), max_value(left(tree)), max_value(right(tree)))

In [10]:
t0 = make_leaf(2)
t1 = make_node(1, make_leaf(3), make_leaf(5)) 
t2 = make_node(3, t1, t0)
value(right(left(t2))) 

5

In [11]:
def inorder(tree):
    if is_empty(tree):
        return []
    return inorder(left(tree)) + [value(tree)] + inorder(right(tree))

In [12]:
# テストケース
print(inorder(t0), inorder(t1), inorder(t2))

[2] [3, 1, 5] [3, 1, 5, 3, 2]


## 課題2

処理の手続きは次の通りである。
1.make_binary_search_treeに配列aを渡す

2.左側の枝＜節点＜右側の枝　という大小関係を成立させるためにaを小さい順でソートする

3.aの真ん中のインデックスの値, 左側の値, 右側の値に分け、それらで木を作る

4.3を再帰的に行い、下の方から順に木を作り、合成して一つの大きな木にする

In [13]:
def make_binary_search_tree(a):
    if not a:
        return EmptyTree
    a.sort()
    mid_index = len(a) // 2
    return make_node(a[mid_index], 
                     make_binary_search_tree(a[:mid_index]), 
                     make_binary_search_tree(a[mid_index + 1:]))

In [14]:
# テストケース
t = make_binary_search_tree([3,1,6,1,7,9,2,0,5])
print(t) 
print(inorder(t))

[3, [1, [1, [0, 'None', 'None'], 'None'], [2, 'None', 'None']], [7, [6, [5, 'None', 'None'], 'None'], [9, 'None', 'None']]]
[0, 1, 1, 2, 3, 5, 6, 7, 9]


## 課題3

 make_question_nodeは質問か回答かの状態typeと、状態として持つ文章textと、yes, noに分岐した場合の値をを辞書型で返す。
 
question_text(qnode),　question_yes(qnode), question_no(qnode)はそのままの実装。

make_answer_nodeは引数に答えの文字列をとり、Answerというtypeを持った答えのノードを返す。つまり質問から回答状態へ移動させる。

answer_textにmake_answer_nodeを放り込むと、答え(text)だけ抽出させる。

is_question_nodeでは質問中か回答中かをtypeから識別する。

これらを作成することで、display関数を適切に動作させることができる。

In [15]:
def make_question_node(question, yes_branch, no_branch):
    return {"type": "question", "text": question, "yes": yes_branch, "no": no_branch}

def question_text(qnode):
    return qnode["text"]

def question_yes(qnode):
    return qnode["yes"]

def question_no(qnode):
    return qnode["no"]

def make_answer_node(answer):
    return {"type": "answer", "text": answer}

def answer_text(anode):
    return anode["text"]

def is_question_node(node):
    return node["type"] == "question"

def display(node):
    if is_question_node(node):
        print("Q: " + question_text(node) + " ? [yn]")
        yn = input()
        if yn.lower()[0] == "y":
            display(question_yes(node))
        else:
            display(question_no(node))
    else:
        print("---")
        print("Answer: " + answer_text(node))

In [16]:
# 関数群のテストケース

root = make_question_node("Do you like animals?",
                          make_question_node("Do you like cats better than dogs?",
                                             make_answer_node("You are a cat person."),
                                             make_answer_node("You are a dog person.")),
                          make_answer_node("You are not an animal lover."))

def test_question_tree(root):

    assert question_text(root) == "Do you like animals?"
    assert question_text(question_yes(root)) == "Do you like cats better than dogs?"
    assert question_text(question_no(root)) == "You are not an animal lover."

    assert answer_text(question_yes(question_yes(root))) == "You are a cat person."
    assert answer_text(question_no(question_yes(root))) == "You are a dog person."
    assert answer_text(question_no(root)) == "You are not an animal lover."

    assert is_question_node(root) == True
    assert is_question_node(question_yes(root)) == True
    assert is_question_node(question_no(root)) == False
    assert is_question_node(question_yes(question_yes(root))) == False
    assert is_question_node(question_no(question_yes(root))) == False

    print("All tests passed for the question tree!")
    
test_question_tree(root)

All tests passed for the question tree!


In [17]:
display(root)

Q: Do you like animals? ? [yn]
y
Q: Do you like cats better than dogs? ? [yn]
n
---
Answer: You are a dog person.


## 課題4

inorderの順番で葉を参照し、その中身に応じて計算を行う。

木の左下から順に3つのノードを取り出し、計算された値を新しい葉に設定する。これを再帰的に行うことで望んだ結果が得られる。

In [18]:
def make_node(value, left=None, right=None):
    return {'value': value, 'left': left, 'right': right}

def calc(tree):
    if type(tree['value']) is int:
        return tree['value']
    left_val = calc(tree['left'])
    right_val = calc(tree['right'])
    if tree['value'] == '+':
        return left_val + right_val
    elif tree['value'] == '-':
        return left_val - right_val
    elif tree['value'] == '*':
        return left_val * right_val
    elif tree['value'] == '/':
        return left_val / right_val

In [19]:
# テストケース
t = make_node("+", make_node("*", make_leaf(3), make_leaf(5)), make_leaf(2))
calc(t)

17

## 課題5

二分木は節点と左右の枝によって表現することができる。そのため、クラスNodeを定義し、その中にvaue, left, rightをまとめて保持させる。

これに伴い、各関数には次の変更を行う。

make_node：辞書型ではなくクラスで定義した変数を割り当てるようにする

value, left, right：引数にインスタンスを想定し、そのvaue, left, rightをそれぞれ返すようにする。

make_binary_search_tree：特に変更なし。

calc：引数にインスタンスを想定し、それに応じて変数の参照を修正する。

これらの変更を行ったあと、関数calcが上手く動くか確かめる。

In [20]:
class Node:
    def __init__(self, value, left=None, right=None):
        self.value = value
        self.left = left
        self.right = right
        
def make_node(num, left, right):
    return Node(num, left, right)

def value(tree):
    return tree.value

def left(tree):
    return tree.left

def right(tree):
    return tree.right

def make_binary_search_tree(a):
    if not a:
        return None
    a.sort()
    mid_index = len(a) // 2
    return make_node(a[mid_index],
                     make_binary_search_tree(a[:mid_index]),
                     make_binary_search_tree(a[mid_index+1:]))

def calc(tree):
    if isinstance(tree.value, int):
        return tree.value
    left_val = calc(tree.left)
    right_val = calc(tree.right)
    if tree.value == '+':
        return left_val + right_val
    elif tree.value == '-':
        return left_val - right_val
    elif tree.value == '*':
        return left_val * right_val
    elif tree.value == '/':
        return left_val / right_val


In [22]:
# テストケース
t = make_node("+", make_node("*", make_leaf(3), make_leaf(5)), make_leaf(2))
calc(t)

17